In [2]:
import numpy as np
import cv2
import random
import pandas as pd
import tensorflow as tf
import pickle
import ast
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from moviepy.editor import VideoFileClip

2023-10-21 22:24:26.746935: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
good = '../dataset/error/good.csv'
misalignedHands = '../dataset/error/misaligned_hands.csv'
narrowLegs = '../dataset/error/narrow_legs.csv'

In [ ]:
df1 = pd.read_csv(good)
df2 = pd.read_csv(misalignedHands)
df3 = pd.read_csv(narrowLegs)

In [ ]:
df = pd.concat([df1, df2], ignore_index=True)
df = pd.concat([df, df3], ignore_index=True)
df = df.drop(columns='Unnamed: 0', axis=1)
df

In [ ]:
newDataset = '../dataset/error/angles.csv'
df.to_csv(newDataset)

Chữa động tác

In [28]:
df = pd.read_csv('../dataset/error/angles.csv')
df = df.drop('Unnamed: 0', axis=1)
# Convert the string representation of Angles into separate columns
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

# Convert the Phase column to integers
df['Phase'] = df['Phase'].astype(int)
df['Correction'] = df['Correction'].astype(int)
df.columns = df.columns.astype(str)
df

,Phase,Correction,Score,0,1,2,3,4,5,6,7
0,1,0,100,1,176,177,0,180,15,177,178
1,1,0,100,2,176,177,0,180,14,178,178
2,1,0,100,2,176,177,0,180,14,179,178
3,1,0,100,2,176,177,0,179,14,179,178
4,1,0,100,2,176,177,0,179,14,179,179
...,...,...,...,...,...,...,...,...,...,...,...
842,2,9,70,3,40,45,0,180,8,179,173
843,2,9,70,3,39,44,0,179,8,179,174
844,2,9,70,4,39,44,0,179,8,179,174
845,2,9,70,3,38,43,0,179,8,180,174


In [35]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']])
labels = np.array(df['Correction'])

In [36]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [37]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, Ytrain)

DecisionTreeClassifier()

In [38]:
Ypred = clf.predict(Xtest)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9882352941176471


In [39]:
Ypred

array([0, 0, 0, 0, 9, 9, 0, 9, 0, 9, 1, 0, 9, 9, 9, 0, 1, 1, 0, 0, 0, 0,
       0, 9, 0, 0, 1, 0, 0, 0, 1, 0, 9, 0, 9, 1, 0, 0, 0, 1, 0, 9, 0, 9,
       1, 0, 0, 9, 0, 9, 1, 1, 9, 0, 0, 9, 0, 0, 9, 9, 0, 9, 0, 0, 9, 0,
       0, 0, 1, 0, 0, 0, 9, 0, 9, 0, 1, 9, 1, 0, 9, 1, 9, 1, 9, 0, 9, 0,
       9, 9, 9, 9, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 9, 9, 0, 0, 1, 0, 0, 1, 9, 9, 0, 0, 9, 9, 0, 0, 1, 0, 0, 9, 9,
       9, 0, 1, 0, 0, 9, 1, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 9, 9, 9, 0, 9, 9, 9, 0, 0, 9, 0, 0, 0, 1, 0])

In [40]:
Ytest

array([0, 0, 0, 0, 9, 9, 0, 9, 0, 9, 1, 0, 9, 9, 9, 0, 1, 1, 0, 0, 0, 0,
       0, 9, 0, 0, 1, 0, 0, 0, 1, 0, 9, 0, 9, 1, 0, 0, 0, 1, 1, 9, 0, 9,
       1, 0, 0, 9, 0, 9, 1, 1, 9, 0, 0, 9, 0, 0, 9, 9, 0, 9, 0, 0, 9, 0,
       0, 0, 1, 0, 0, 0, 9, 0, 9, 0, 1, 9, 1, 0, 9, 1, 9, 1, 9, 0, 9, 0,
       9, 9, 9, 9, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 9, 9, 0, 0, 1, 0, 0, 1, 9, 9, 0, 0, 9, 9, 0, 0, 1, 0, 0, 9, 9,
       9, 0, 1, 0, 0, 9, 1, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 9, 9, 9, 0, 9, 9, 9, 0, 0, 9, 0, 0, 0, 1, 0])

In [ ]:
Xtest = Xtest.sort_index()
Ytest = Ytest.sort_index()

In [ ]:
clf.predict(Xtest.loc[5:25])

In [ ]:
Xtest.loc[5:25]

In [ ]:
Ytest.loc[5:25]

Tính gia tốc, vận tốc

In [7]:
# Đếm số giây của video
videoPath = '../assets/sample-video/sample-video01.mp4'
video = VideoFileClip(videoPath)
duration = video.duration
duration

8.44

In [8]:
# Đếm số frame của video
cap = cv2.VideoCapture(videoPath)
frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frameCount

253

In [9]:
# Tính khoảng thời gian giữa 2 frame
deltaT = duration / (frameCount - 1)
deltaT

0.03349206349206349

In [3]:
df = pd.read_csv('../dataset/standard/angles01.csv')
df = df.drop('Unnamed: 0', axis=1)
df

,Angles,Phase,Score
0,"[1, 176, 177, 0, 180, 15, 177, 178]",Down,100
1,"[2, 176, 177, 0, 180, 14, 178, 178]",Down,100
2,"[2, 176, 177, 0, 180, 14, 179, 178]",Down,100
3,"[2, 176, 177, 0, 179, 14, 179, 178]",Down,100
4,"[2, 176, 177, 0, 179, 14, 179, 179]",Down,100
...,...,...,...
248,"[3, 175, 175, 0, 179, 14, 179, 179]",Down,100
249,"[3, 175, 175, 0, 179, 13, 179, 179]",Down,100
250,"[3, 175, 175, 0, 179, 14, 179, 179]",Down,100
251,"[3, 175, 175, 0, 179, 14, 179, 179]",Down,100


In [4]:
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

In [5]:
df.columns = df.columns.astype(str)

In [6]:
leftArms = df['1']

In [10]:
t = []
for i in range(len(leftArms)):
    t.append(deltaT * i)

In [11]:
x = np.array(leftArms)
t = np.array(t)

# Tính đạo hàm của mảng theo t
dx_dt = np.gradient(x, t)
dx_dt = dx_dt.astype(int)
print(dx_dt[0:75])

[    0     0     0     0     0     0     0     0     0     0     0   -14
   -14   -14   -14   -14   -14   -14   -14   -14   -14   -14   -59   -74
   -59   -59  -119  -298  -612 -1209 -1164  -507  -223  -134  -104   -59
   -29   -14   -14   -14   -14   -14   -14   -14   -14   -14     0     0
     0   -14   -14     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0    14    14     0
     0    14    14]


In [12]:
t = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
x = np.array([df['1'][0],
              df['1'][30],
              df['1'][60],
              df['1'][90],
              df['1'][120],
              df['1'][150],
              df['1'][180],
              df['1'][210],
              df['1'][240],])
dx_dt = np.gradient(x, t)
dx_dt = dx_dt.astype(int)
dx_dt

array([-116,  -84,   11,   83,   44,  -83,  -20,   83,   45])

Chấm điểm

In [13]:
df = pd.read_csv('../dataset/error/angles.csv')
df = df.drop('Unnamed: 0', axis=1)
# Convert the string representation of Angles into separate columns
df['Angles'] = df['Angles'].apply(lambda x: [int(val) for val in x.strip('[]').split(', ')])

# Split the Angles list into separate columns
df = pd.concat([df, df['Angles'].apply(pd.Series)], axis=1)
df.drop('Angles', axis=1, inplace=True)

# Convert the Phase column to integers
df['Phase'] = df['Phase'].astype(int)
df['Score'] = df['Score'].astype(int)
df.columns = df.columns.astype(str)
df

,Phase,Correction,Score,0,1,2,3,4,5,6,7
0,1,0,100,1,176,177,0,180,15,177,178
1,1,0,100,2,176,177,0,180,14,178,178
2,1,0,100,2,176,177,0,180,14,179,178
3,1,0,100,2,176,177,0,179,14,179,178
4,1,0,100,2,176,177,0,179,14,179,179
...,...,...,...,...,...,...,...,...,...,...,...
842,2,9,70,3,40,45,0,180,8,179,173
843,2,9,70,3,39,44,0,179,8,179,174
844,2,9,70,4,39,44,0,179,8,179,174
845,2,9,70,3,38,43,0,179,8,180,174


In [ ]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']])
labels = np.array(df['Score'])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

LR

In [ ]:
model = LinearRegression()
reg = model.fit(Xtrain, Ytrain)
Ypred = reg.predict(Xtest)
Ypred = Ypred.astype(int)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
Ypred

In [ ]:
Ytest

SVR

In [ ]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']])
labels = np.array(df['Score'])
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
svr = svm.SVR(kernel='linear')
svr.fit(Xtrain, Ytrain)
Ypred = svr.predict(Xtest)
Ypred = Ypred.astype(int)
accuracy = accuracy_score(Ytest, Ypred)
print(f"Accuracy: {accuracy * 100:.2f}%")

In [ ]:
Ypred

In [ ]:
Ytest

TensorFlow

In [14]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']].values)
labels = np.array(df['Score'].values)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [15]:
# Chuẩn hoá dữ liệu
scaler = StandardScaler()
Xtrain = scaler.fit_transform(Xtrain)
Xtest = scaler.transform(Xtest)

In [16]:
# Xây dựng mô hình mạng neural
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(9,)),  # Số lượng đặc trưng đầu vào
    tf.keras.layers.Dense(64, activation='relu'),  # Lớp ẩn với 64 đơn vị ẩn
    tf.keras.layers.Dense(32, activation='relu'),  # Lớp ẩn với 32 đơn vị ẩn
    tf.keras.layers.Dense(1)  # Lớp đầu ra với 1 đơn vị (Regression)
])

In [17]:
# Biên dịch mô hình
model.compile(optimizer='adam', loss='mean_squared_error')

# Huấn luyện mô hình
model.fit(Xtrain, Ytrain, epochs=50, batch_size=32, validation_data=(Xtest, Ytest))

# Đánh giá hiệu suất của mô hình
loss = model.evaluate(Xtest, Ytest)
print(f"Loss on test data: {loss}")

Epoch 1/50
22/22 [==============================] - 1s 11ms/step - loss: 7925.0474 - val_loss: 7596.8301
Epoch 2/50
22/22 [==============================] - 0s 3ms/step - loss: 7610.8843 - val_loss: 7206.6646
Epoch 3/50
22/22 [==============================] - 0s 4ms/step - loss: 7126.2856 - val_loss: 6592.9038
Epoch 4/50
22/22 [==============================] - 0s 4ms/step - loss: 6372.8057 - val_loss: 5664.1191
Epoch 5/50
22/22 [==============================] - 0s 4ms/step - loss: 5268.7256 - val_loss: 4393.8750
Epoch 6/50
22/22 [==============================] - 0s 3ms/step - loss: 3870.3770 - val_loss: 2920.4343
Epoch 7/50
22/22 [==============================] - 0s 3ms/step - loss: 2388.1936 - val_loss: 1558.3413
Epoch 8/50
22/22 [==============================] - 0s 3ms/step - loss: 1185.7357 - val_loss: 658.6196
Epoch 9/50
22/22 [==============================] - 0s 3ms/step - loss: 502.4116 - val_loss: 297.5081
Epoch 10/50
22/22 [==============================] - 0s 3ms/step -

In [18]:
# Sử dụng mô hình để dự đoán
predictions = model.predict(Xtest)

6/6 [==============================] - 0s 2ms/step


In [19]:
Ypred = predictions.astype(int)
Ypred = predictions.reshape(-1,)
Ypred

array([ 97.19758 ,  94.48356 ,  93.52134 ,  90.514084,  79.15575 ,
        68.52155 , 109.63765 ,  71.18899 ,  96.46184 ,  61.861553,
        80.09524 , 106.57457 ,  76.17919 ,  77.16708 ,  79.567955,
        91.92383 ,  83.939964,  81.356926,  81.51818 ,  96.30506 ,
        91.11336 ,  82.24435 ,  90.22245 ,  69.52622 ,  97.40574 ,
       103.42819 ,  70.525314, 100.904884,  87.936714, 104.42005 ,
        84.637856, 101.5479  ,  78.79348 ,  96.68324 ,  77.19543 ,
        78.55315 ,  99.75782 ,  95.942116,  96.32139 ,  75.83257 ,
        74.44317 ,  81.211586,  94.32792 ,  84.804115,  75.74185 ,
        96.38497 , 105.07697 ,  78.01428 ,  87.53708 ,  75.927086,
        76.86663 ,  72.73827 ,  79.89469 , 102.3553  ,  94.836655,
        68.1091  ,  99.90971 ,  85.19834 ,  85.66445 ,  80.22142 ,
        94.59472 ,  73.00095 , 104.57085 , 103.98109 ,  79.11744 ,
       101.85852 ,  89.81889 ,  93.10436 ,  81.0894  ,  97.19758 ,
        95.27648 , 105.07697 ,  86.54418 ,  96.041374,  71.229

In [20]:
Ytest

array([100,  80, 100, 100,  70,  70, 100,  70, 100,  70,  80, 100,  80,
        80,  80, 100,  70,  80,  85,  90, 100, 100, 100,  80,  90, 100,
        70, 100,  90, 100,  70, 100,  80, 100,  80,  70, 100, 100, 100,
        80,  85,  80, 100,  80,  70, 100, 100,  80,  95,  80,  80,  80,
        80, 100, 100,  80, 100,  95,  80,  80, 100,  80, 100, 100,  80,
       100, 100,  90,  80, 100, 100, 100,  80,  95,  60, 100,  70,  80,
        80, 100,  70,  85,  70,  70,  80, 100,  70, 100,  80,  80,  80,
        70, 100, 100, 100, 100,  90,  70, 100, 100,  95,  70, 100, 100,
       100,  70, 100,  80, 100, 100,  90,  70,  80, 100,  95,  80,  90,
        95,  80,  70,  80, 100, 100,  80,  80,  90, 100,  70, 100, 100,
        80,  80,  80, 100,  80, 100, 100,  80,  70, 100, 100,  90,  95,
       100,  80, 100, 100,  90, 100, 100,  90,  80,  90,  90,  80,  80,
        80,  80, 100,  80,  60,  70, 100, 100,  80,  90,  95, 100,  70,
       100])

In [21]:
# Tính MSE
mse = mean_squared_error(Ytest, Ypred)
print(f"Mean Squared Error (MSE): {mse}")

# Tính RMSE
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Mean Squared Error (MSE): 42.02472670277117
Root Mean Squared Error (RMSE): 6.482648124244533


In [22]:
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')

Accuracy: 54.12%


Random Forest

In [23]:
features = np.array(df[['Phase', '0', '1', '2', '3', '4', '5', '6', '7']].values)
labels = np.array(df['Score'].values)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(features, labels, test_size=0.2, random_state=42)

In [24]:
# Tạo mô hình Random Forest Regression
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)  # Bạn có thể tùy chỉnh số cây và các siêu tham số khác

# Huấn luyện mô hình trên tập huấn luyện
rf_model.fit(Xtrain, Ytrain)

# Dự đoán trên tập kiểm tra
Ypred = rf_model.predict(Xtest)

# Đánh giá mô hình
mse = mean_squared_error(Ytest, Ypred)
r2 = r2_score(Ytest, Ypred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")

Mean Squared Error (MSE): 3.1750045955882356
R-squared (R2): 0.976485214386392


In [25]:
Ypred = Ypred.astype(int)
Ypred

array([100,  91,  99, 100,  75,  70, 100,  75, 100,  71,  81, 100,  80,
        79,  80, 100,  66,  80,  85,  90,  97,  99,  96,  80,  91,  98,
        71,  99,  88, 100,  67, 100,  80, 100,  80,  67, 100,  98, 100,
        80,  84,  80, 100,  80,  71, 100, 100,  80,  95,  79,  80,  80,
        80, 100,  99,  80, 100,  94,  79,  80,  99,  79,  98, 100,  80,
        97,  96,  90,  78, 100, 100, 100,  80,  94,  61, 100,  70,  80,
        79,  98,  70,  81,  69,  71,  79, 100,  75, 100,  76,  80,  73,
        74, 100, 100,  99, 100,  90,  72, 100, 100,  95,  71, 100, 100,
       100,  73, 100,  80,  99, 100,  90,  70,  80,  99,  88,  80,  90,
        96,  79,  70,  80, 100, 100,  80,  80,  90,  97,  72,  99, 100,
        80,  80,  80,  98,  79, 100, 100,  80,  70,  99, 100,  90,  90,
        97,  80, 100, 100,  90, 100, 100,  90,  80,  88,  90,  80,  80,
        79,  79, 100,  80,  62,  70, 100, 100,  79,  92,  95, 100,  72,
       100])

In [26]:
Ytest

array([100,  80, 100, 100,  70,  70, 100,  70, 100,  70,  80, 100,  80,
        80,  80, 100,  70,  80,  85,  90, 100, 100, 100,  80,  90, 100,
        70, 100,  90, 100,  70, 100,  80, 100,  80,  70, 100, 100, 100,
        80,  85,  80, 100,  80,  70, 100, 100,  80,  95,  80,  80,  80,
        80, 100, 100,  80, 100,  95,  80,  80, 100,  80, 100, 100,  80,
       100, 100,  90,  80, 100, 100, 100,  80,  95,  60, 100,  70,  80,
        80, 100,  70,  85,  70,  70,  80, 100,  70, 100,  80,  80,  80,
        70, 100, 100, 100, 100,  90,  70, 100, 100,  95,  70, 100, 100,
       100,  70, 100,  80, 100, 100,  90,  70,  80, 100,  95,  80,  90,
        95,  80,  70,  80, 100, 100,  80,  80,  90, 100,  70, 100, 100,
        80,  80,  80, 100,  80, 100, 100,  80,  70, 100, 100,  90,  95,
       100,  80, 100, 100,  90, 100, 100,  90,  80,  90,  90,  80,  80,
        80,  80, 100,  80,  60,  70, 100, 100,  80,  90,  95, 100,  70,
       100])

In [27]:
wrongPre = 0
totalPre = len(Ypred)
eps = 5
for i in range(totalPre):
    if abs(Ypred[i] - Ytest[i]) > 5:
        wrongPre += 1

print(f'Accuracy: {(1 - wrongPre / totalPre) * 100:.2f}%')

Accuracy: 98.24%
